# YOLOv7 Model Training

### importations

In [ ]:
import os
import glob as glob
import matplotlib.pyplot as plt
import random
import cv2

### Downloading yolov7 model with all files

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git

### installing all required libraries for yolov7

In [ ]:
%cd yolov7
!pip install -r requirements.txt
!pip uninstall wandb -y

### downloading the tiny model to train on custom dataset
#### you can use the regular model (https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt) but it will be larger

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt

In [ ]:
%mkdir config

### creating custom config for custom data training

In [ ]:
%%writefile config/pothole.yaml
train: ../dataset/images/train 
val: ../dataset/images/valid
test: ../dataset/images/test

# Classes
nc: 1  # number of classes
names: ['pothole']  # class names

### changing class number because we used only 1

In [ ]:
old = open("yolov7/cfg/training/yolov7-tiny.yaml","r").readlines()
old[1] = "nc: 1  # number of classes\n"
new = open("yolov7/cfg/training/yolov7-tiny.yaml","w")
for line in old:
    new.writelines(line)
new.close()

### Model Training & Saving

In [ ]:
!python train.py --epochs 100 --workers 4 --device 0 --batch-size 4 --data data/pothole.yaml --img 640 640 --cfg config/pothole.yaml --weights 'yolov7-tiny.pt' --name pothole --hyp data/hyp.scratch.tiny.yaml

___

---

# CNN Model Training

In [ ]:
%cd ..

### importations

In [ ]:
import matplotlib.image as mpimg
   
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from keras.models import load_model
from keras import layers
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from sklearn.model_selection import train_test_split
import PIL.Image as Image
import tensorflow as tf
import pandas as pd
import numpy as np
import pathlib
import PIL
import warnings
warnings.filterwarnings('ignore')

### datasets

In [ ]:
Train_data_dir = "DataSet2/Train/"
Validate_data_dir = "DataSet2/Valid/"
Train_data_dir = pathlib.Path(Train_data_dir)
Validate_data_dir = pathlib.Path(Validate_data_dir)

### Basic configuration

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  Train_data_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  Validate_data_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
num_classes = len(class_names)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalization_layer = layers.Rescaling(1./255)

### Preprocessing data to avoid overfitting

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

### Building Model

In [ ]:
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
epochs = 15

### Model Training & Saving

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)
model.save('CNN_Model.h5')